Pierre Navaro - [Institut de Recherche Mathématique de Rennes](https://irmar.univ-rennes1.fr) - [CNRS](http://www.cnrs.fr/)

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import scipy.fftpack as sf
import scipy.linalg as sl
import numpy as np

In [3]:
%env CC=gcc-7

env: CC=gcc-7


# f2py
f2py is a part of Numpy and there are three ways to wrap Fortran with Python :
- Write some fortran subroutines and just run f2py to create Python modules.
- Insert special f2py directives inside Fortran source for complex wrapping.
- Write a interface file (.pyf) to wrap Fortran files without changing them. f2py automatically generate the pyf template file that can be modified. 

# Simple Fortran subroutine to compute norm
   
### Fortran 90/95 free format

In [4]:
%%file euclidian_norm.f90
subroutine euclidian_norm (a, b, c)
real(8), intent(in) :: a, b
real(8), intent(out) :: c 
c =	sqrt (a*a+b*b) 
end subroutine euclidian_norm

Overwriting euclidian_norm.f90


### Fortran 77 fixed format

In [5]:
%%file euclidian_norm.f
      subroutine euclidian_norm (a, b, c)
      real*8 a,b,c
Cf2py intent(out) c
      c = sqrt (a*a+b*b) 
      end 

Overwriting euclidian_norm.f


# Build extension module with f2py program


In [6]:
!f2py -c euclidian_norm.f90 -m vect  --fcompiler=gnu95 --f90flags=-O3

running build
running config_cc
unifing config_cc, config, build_clib, build_ext, build commands --compiler options
running config_fc
unifing config_fc, config, build_clib, build_ext, build commands --fcompiler options
running build_src
build_src
building extension "vect" sources
f2py options: []
f2py:> /var/folders/56/lhgl67bx4qj6tvjx41b1wgdr0000gn/T/tmp3zahd6c_/src.macosx-10.12-x86_64-3.6/vectmodule.c
creating /var/folders/56/lhgl67bx4qj6tvjx41b1wgdr0000gn/T/tmp3zahd6c_/src.macosx-10.12-x86_64-3.6
Reading fortran codes...
	Reading file 'euclidian_norm.f90' (format:free)
Post-processing...
	Block: vect
			Block: euclidian_norm
Post-processing (stage 2)...
Building modules...
	Building module "vect"...
		Constructing wrapper function "euclidian_norm"...
		  c = euclidian_norm(a,b)
	Wrote C/API module "vect" to file "/var/folders/56/lhgl67bx4qj6tvjx41b1wgdr0000gn/T/tmp3zahd6c_/src.macosx-10.12-x86_64-3.6/vectmodule.c"
  adding '/var/folders/56/lhgl67bx4qj6tvjx41b1wgdr0000gn/T/tmp3zahd6c

## Use the extension module in Python

In [7]:
import vect
c = vect.euclidian_norm(3,4)
c

5.0

In [8]:
print(vect.euclidian_norm.__doc__) # Docstring is automatically generate

c = euclidian_norm(a,b)

Wrapper for ``euclidian_norm``.

Parameters
----------
a : input float
b : input float

Returns
-------
c : float



# Fortran magic 

- Jupyter extension that help to use fortran code in an interactive session.
- It adds a %%fortran cell magic that compile and import the Fortran code in the cell, using F2py.
- The contents of the cell are written to a .f90 file in the directory IPYTHONDIR/fortran using a filename with the hash of the code. This file is then compiled. The resulting module is imported and all of its symbols are injected into the user's namespace.

[Documentation](http://nbviewer.jupyter.org/github/mgaitan/fortran_magic/blob/master/documentation.ipynb)

In [9]:
%load_ext fortranmagic

/usr/local/lib/python3.6/site-packages/fortranmagic.py:147: UserWarning: get_ipython_cache_dir has moved to the IPython.paths module since IPython 4.0.
  self._lib_dir = os.path.join(get_ipython_cache_dir(), 'fortran')


# F2py directives
- F2PY introduces also some extensions to Fortran 90/95 language specification that help designing Fortran to Python interface, make it more “Pythonic”.
- If editing Fortran codes is acceptable, these specific attributes can be inserted directly to Fortran source codes. Special comment lines are ignored by Fortran compilers but F2PY interprets them as normal lines.



In [10]:
%%fortran 
subroutine euclidian_norm(a,c,n) 
integer :: n 
real(8),dimension(n),intent(in) :: a
!f2py optional , depend(a) :: n=len(a)
real(8),intent(out) :: c 
real(8) :: sommec 
integer :: i
sommec = 0 
do i=1,n
   sommec=sommec+a( i )*a( i ) 
end do
c=sqrt (sommec) 
end subroutine euclidian_norm

In [11]:
a=[2,3,4]  # Python list
type(a)

list

In [12]:
euclidian_norm(a)

5.385164807134504

In [13]:
a=np.arange(2,5)  # numpy array
type(a)

numpy.ndarray

In [14]:
euclidian_norm(a)

5.385164807134504

In [15]:
print(euclidian_norm.__doc__) # Documentation

c = euclidian_norm(a,[n])

Wrapper for ``euclidian_norm``.

Parameters
----------
a : input rank-1 array('d') with bounds (n)

Other Parameters
----------------
n : input int, optional
    Default: len(a)

Returns
-------
c : float



# F2py directives
- `optional`: The corresponding argument is moved to the end.
- `required`: This is default. Use it to disable automatic optional setting.
- `intent(in | inout | out | hide)` , `intent(in)` is the default.
- `intent(out)` is implicitly translated to `intent(out,hide) `.
- `intent(copy)` and `intent(overwrite)` control changes for input arguments.
- `check` performs some assertions, it is often automatically generated.
- `depend`:  f2py detects cyclic dependencies.
- `allocatable, parameter`
- `intent(callback), external`: for function as arguments.
- `intent(c)` C-type argument , array or function.
- C expressions: `rank, shape, len, size, slen`.


# Callback

You can call a python function inside your fortran code


In [16]:
%%fortran
subroutine sum_f (f ,n,s) 
  !Compute sum(f(i), i=1,n) 
  external f 
  integer, intent(in) :: n 
  real, intent(out) ::s
  s = 0.0 
  do i=1,n
    s=s+f(i)
  end do 
end subroutine sum_f

In [17]:
def fonction(i) : # python function
    return i*i

sum_f(fonction,3) 

14.0

In [18]:
sum_f(lambda x :x**3,3) # lambda function

36.0

# Fortran arrays and Numpy arrays

Let's see how to pass numpy arrays to fortran subroutine.


In [19]:
%%fortran --extra "-DF2PY_REPORT_ON_ARRAY_COPY=1"
subroutine push( positions, velocities, dt, n)
  integer, intent(in) :: n
  real(8), intent(in) :: dt
  real(8), dimension(n,3), intent(in) :: velocities
  real(8), dimension(n,3) :: positions
  do i = 1, n
    positions(i,:) = positions(i,:) + dt*velocities(i,:)
  end do
end subroutine push


In [20]:
positions = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
velocities = [[0, 1, 2], [0, 3, 2], [0, 1, 3]]

In [21]:
push(positions, velocities, 0.1)
positions # memory is not updated because we used C memory storage


[[0, 0, 0], [0, 0, 0], [0, 0, 0]]

During execution, the message "created an array from object" is displayed, because a copy of is made when passing multidimensional array to fortran subroutine.

In [22]:
positions = np.array(positions, dtype='f8', order='F')
push(positions, velocities, 0.1)
positions # the memory is updated

array([[ 0. ,  0.1,  0.2],
       [ 0. ,  0.3,  0.2],
       [ 0. ,  0.1,  0.3]])

# Signature file

This file contains descriptions of wrappers to Fortran or C functions, also called as signatures of the functions. F2PY can create initial signature file by scanning Fortran source codes and catching all relevant information needed to create wrapper functions.

```bash
f2py vector.f90 -h vector.pyf
```
- vector.pyf

```fortran
!    -*- f90 -*-
! Note: the context of this file is case sensitive.

subroutine euclidian_norm(a,c,n) ! in vector.f90
    real(kind=8) dimension(n),intent(in) :: a
    real(kind=8) intent(out) :: c
    integer optional,check(len(a)>=n),depend(a) :: n=len(a)
end subroutine euclidian_norm

! This file was auto-generated with f2py (version:2).
! See http://cens.ioc.ee/projects/f2py2e/
```


# Wrap lapack function dgemm  with f2py

- Generate the signature file

In [23]:
%rm -f dgemm.f dgemm.pyf
!wget http://ftp.mcs.anl.gov/pub/MINPACK-2/blas/dgemm.f

--2017-11-11 21:58:00--  http://ftp.mcs.anl.gov/pub/MINPACK-2/blas/dgemm.f
Resolving ftp.mcs.anl.gov... 140.221.6.23
Connecting to ftp.mcs.anl.gov|140.221.6.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9851 (9.6K) [text/plain]
Saving to: 'dgemm.f'

dgemm.f             100%[===================>]   9.62K  --.-KB/s    in 0.004s  

2017-11-11 21:58:00 (2.23 MB/s) - 'dgemm.f' saved [9851/9851]



In [24]:
# %load dgemm.f

In [25]:
!f2py -m mylapack --overwrite-signature -h dgemm.pyf dgemm.f

Reading fortran codes...
	Reading file 'dgemm.f' (format:fix,strict)
Post-processing...
	Block: mylapack
			Block: dgemm
Post-processing (stage 2)...
Saving signatures to file "./dgemm.pyf"


```fortran
!    -*- f90 -*-
! Note: the context of this file is case sensitive.

python module mylapack ! in 
    interface  ! in :mylapack
        subroutine dgemm(transa,transb,m,n,k,alpha,a,lda,b,ldb,beta,c,ldc) ! in :mylapack:dgemm.f
            character*1 :: transa
            character*1 :: transb
            integer :: m
            integer :: n
            integer :: k
            double precision :: alpha
            double precision dimension(lda,*) :: a
            integer, optional,check(shape(a,0)==lda),depend(a) :: lda=shape(a,0)
            double precision dimension(ldb,*) :: b
            integer, optional,check(shape(b,0)==ldb),depend(b) :: ldb=shape(b,0)
            double precision :: beta
            double precision dimension(ldc,*) :: c
            integer, optional,check(shape(c,0)==ldc),depend(c) :: ldc=shape(c,0)
        end subroutine dgemm
    end interface 
end python module mylapack

! This file was auto-generated with f2py (version:2).
! See http://cens.ioc.ee/projects/f2py2e/
```

In [26]:
!f2py -c dgemm.pyf -llapack

running build
running config_cc
unifing config_cc, config, build_clib, build_ext, build commands --compiler options
running config_fc
unifing config_fc, config, build_clib, build_ext, build commands --fcompiler options
running build_src
build_src
building extension "mylapack" sources
creating /var/folders/56/lhgl67bx4qj6tvjx41b1wgdr0000gn/T/tmpp5irpzo7/src.macosx-10.12-x86_64-3.6
f2py options: []
f2py: dgemm.pyf
Reading fortran codes...
	Reading file 'dgemm.pyf' (format:free)
Post-processing...
	Block: mylapack
			Block: dgemm
Post-processing (stage 2)...
Building modules...
	Building module "mylapack"...
		Constructing wrapper function "dgemm"...
getarrdims:warning: assumed shape array, using 0 instead of '*'
getarrdims:warning: assumed shape array, using 0 instead of '*'
getarrdims:warning: assumed shape array, using 0 instead of '*'
		  dgemm(transa,transb,m,n,k,alpha,a,b,beta,c,[lda,ldb,ldc])
	Wrote C/API module "mylapack" to file "/var/folders/56/lhgl67bx4qj6tvjx41b1wgdr0000gn/T/t

In [27]:
import numpy as np
import mylapack
a = np.array([[7,8],[3,4],[1,2]])
b = np.array([[1,2,3],[4,5,6]])
print("a=",a) 
print("b=",b)
assert a.shape[1] == b.shape[0]
c = np.zeros((a.shape[0],b.shape[1]),'d',order='F')
mylapack.dgemm('N','N',a.shape[0],b.shape[1],a.shape[1],1.0,a,b,1.0,c)
print(c)
np.all(c == a @ b) # check with numpy matrix multiplication 

a= [[7 8]
 [3 4]
 [1 2]]
b= [[1 2 3]
 [4 5 6]]
[[ 39.  54.  69.]
 [ 19.  26.  33.]
 [  9.  12.  15.]]


True

```fortran
python module mylapack2 ! in 
    interface  ! in :mylapack
        subroutine dgemm(transa,transb,m,n,k,alpha,a,lda,b,ldb,beta,c,ldc)
            character, optional :: transa = 'N'
            character, optional :: transb = 'N'
            integer, optional :: m = shape(a,0)
            integer, optional :: n = shape(b,1)
            integer, optional, check(shape(b,0)==k) :: k = shape(a,1)
            double precision, optional :: alpha = 1.
            double precision dimension(lda,*) :: a
            integer optional,check(shape(a,0)==lda),depend(a) :: lda=shape(a,0)
            double precision dimension(ldb,*) :: b
            integer optional,check(shape(b,0)==ldb),depend(b) :: ldb=shape(b,0)
            double precision, optional :: beta = 1.
            double precision dimension(shape(a,0),shape(b,1)), intent(out) :: c
            integer optional :: ldc=shape(a,0)
        end subroutine dgemm
    end interface 
end python module mylapack2
```

# Création du module python

In [28]:
!f2py -c solutions/dgemm.pyf -llapack --f90flags=-O3

running build
running config_cc
unifing config_cc, config, build_clib, build_ext, build commands --compiler options
running config_fc
unifing config_fc, config, build_clib, build_ext, build commands --fcompiler options
running build_src
build_src
building extension "mylapack2" sources
creating /var/folders/56/lhgl67bx4qj6tvjx41b1wgdr0000gn/T/tmpka_uxhnf/src.macosx-10.12-x86_64-3.6
creating /var/folders/56/lhgl67bx4qj6tvjx41b1wgdr0000gn/T/tmpka_uxhnf/src.macosx-10.12-x86_64-3.6/solutions
f2py options: []
f2py: solutions/dgemm.pyf
Reading fortran codes...
	Reading file 'solutions/dgemm.pyf' (format:free)
Post-processing...
	Block: mylapack2
			Block: dgemm
Post-processing (stage 2)...
Building modules...
	Building module "mylapack2"...
		Constructing wrapper function "dgemm"...
getarrdims:warning: assumed shape array, using 0 instead of '*'
getarrdims:warning: assumed shape array, using 0 instead of '*'
		  c = dgemm(a,b,[transa,transb,m,n,k,alpha,lda,ldb,beta,ldc])
	Wrote C/API module "

In [29]:
import mylapack2
a = np.array([[7,8],[3,4],[1,2]])
b = np.array([[1,2,3],[4,5,6]])
c = mylapack2.dgemm(a,b)
np.all( c == a @ b)

True

## Check performance between numpy and mylapack

In [30]:
a = np.random.random((512,128))
b = np.random.random((128,512))

In [31]:
%timeit c = mylapack2.dgemm(a,b)

954 µs ± 83.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [32]:
%timeit c = a @ b

649 µs ± 4.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


**Fortran arrays allocated in a subroutine share same memory in Python**

In [33]:
%%fortran
module f90module
   implicit none
   real(8), dimension(:),   allocatable :: farray
contains
   subroutine init( n ) !Allocation du tableau farray
   integer, intent(in) :: n
   allocate(farray(n))
   end subroutine init
end module f90module

In [34]:
f90module.init(10)
len(f90module.farray)

10

**Numpy arrays allocated in Python passed to Fortran are already allocated**

In [35]:
%%fortran
module f90module
   implicit none
   real(8), dimension(:),   allocatable :: farray
contains
   subroutine test_array( allocated_flag, array_size )
   logical, intent(out) :: allocated_flag
   integer, intent(out) :: array_size
   allocated_flag = allocated(farray)
   array_size = size(farray)
   end subroutine test_array
end module f90module

In [36]:
f90module.farray = np.random.rand(10).astype(np.float64)
f90module.test_array()

(1, 10)

# Derived type

In [39]:
%%fortran
module mesh
implicit none
type :: geometry
   real(8) :: x0, x1, dx    ! coordinates of origin and grid size
   integer :: nx               ! number of grid points
   real(8), dimension(:), pointer :: xgrid  ! coordinates of points
end type geometry

contains

subroutine create(geom,x0,nx,dx) 
!f2py integer, intent(out) :: geom
type(geometry), pointer :: geom
real(8), intent(in) :: x0, dx
integer, intent(in) :: nx
integer :: i   
allocate(geom)
geom%x0=x0
geom%x1=x0+nx*dx
geom%dx=dx
geom%nx=nx
allocate(geom%xgrid(nx))
do i=1,nx
   geom%xgrid(i)=geom%x0+(i-1)*geom%dx
end do
end subroutine create

subroutine view(this)
!f2py integer, intent(in) :: this
type(geometry),pointer :: this
print*, 'nx = ', this%nx
print*, this%x0,this%x1
print*, this%xgrid(:)
end subroutine view

end module mesh

In [ ]:
geom = mesh.create(0.0,10,0.1)
mesh.view(geom)

# f2py + OpenMP



In [58]:
import os
os.environ['CC'] = 'gcc-7' # for mac users

In [37]:
%%fortran 
subroutine hello( )
integer :: i
do i = 1, 4
   call sleep(1)
end do
end subroutine

In [38]:
%%time
hello()

CPU times: user 10.8 ms, sys: 41.1 ms, total: 51.9 ms
Wall time: 4.01 s


In [39]:
%%fortran --f90flags "-fopenmp" --extra "-L/usr/local/lib -lgomp"
subroutine hello_omp( )

integer :: i
!$OMP PARALLEL PRIVATE(I)
!$OMP DO 
do i = 1, 4
   call sleep(1)
end do
!$OMP END DO
!$OMP END PARALLEL

end subroutine


In [40]:
%%time
hello_omp()

CPU times: user 3 ms, sys: 9.91 ms, total: 12.9 ms
Wall time: 1.01 s


# Conclusions

- Easy to use, it works with modern fortran, legacy fortran and also C.
- Works with common and modules and arrays dynamically allocated.
- Python function callback can be very useful combined with Sympy
- Documentation is  automatically generated
- All fortran compilers are supported: GNU, Portland, Sun, Intel,...
- F2py is integrated in numpy library.

## cons 
- Derived types and fortran pointers are not well supported.
- Absolutely not compatible with fortran 2003-2008 new features (classes)
- f2py is maintained but not really improved. Development is stopped.

# distutils

## setup.py
```python
from numpy.distutils.core import Extension, setup
ext1 = Extension(name = 'scalar',
                 sources = ['scalar.f'])
ext2 = Extension(name = 'fib2',
                 sources = ['fib2.pyf','fib1.f'])

setup(name = 'f2py_example', ext_modules = [ext1,ext2])
```
Compilation
```bash
python3 setup.py build_ext --inplace
```

# Ressources
- [Transparents E. Sonnendrücker](http://calcul.math.cnrs.fr/Documents/Journees/dec2006/python-fortran.pdf)
- [SciPy](http://www.scipy.org/F2py)
- [Documentation Sagemath](http://www.sagemath.org/doc/numerical_sage/f2py.html) 
- Hans Petter Langtangen.*Python Scripting for Computational Science*. Springer 2004
